<a href="https://colab.research.google.com/github/hereagain-Y/TF_tutorial/blob/main/VAE_615.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv("/content/drive/MyDrive/DL/VAE/rep_00148.tsv",
            sep="\t")
comp_seq=data["amino_acid"].tolist()
seq='IHTGCYRTQYNQGNK'
print(comp_seq)


['WPPHRNQASTKCLNNMQ', 'PLQEDGMNNDH', 'MFRQLIPEIIC', 'TLRQMCHKHKVV', 'SNRWDGRSIWHVDKAYH', 'EHMCIQGNFLAGLVS', 'YITNHHNVPLIAFWY', 'NYLHGNGSWSVSGIPF', 'YIAFWMIDMG', 'LAEEMGRCPI', 'PCRWRHCEHTL', 'CEPWAGGVKYK', 'KDQGVWCHNLPDKYI', 'VSHCDKCREGQAISDMEHS', 'QCKFCTRDCWDSG', 'DYGGVGFTNCMY', 'VNDEMDWETQHGGVHQKT', 'RDCYQIKPHDINPTYE', 'SMKVQDFPTPSC', 'RWCLVKRIGHCMD', 'HMLQFHYQRGT', 'DPVECCWHLQLE', 'MTHRANSCGSK', 'TETIFIHWFSNIRQE', 'YVKLKTPLKGYFRYYIE', 'ETGMTHNWEVWSLHDYI', 'YYQIRGYRKDVMM', 'AHGNMAVKHSRDKPTWYM', 'TKLPRVTFWICGQPER', 'QSMSTMFAMKKQ', 'WCLHEQDSTWRWVAVNLE', 'YLYEATLMKR', 'GSCHIPMKYCWHTRGCFHN']


In [3]:
ONE_HOT = {'A': 0, 'C': 1, 'D': 2, 'E': 3, 'F': 4, 'G': 5, 'H': 6, 'I': 7, 'K': 8, 'L': 9,
                   'M': 10, 'N': 11, 'P': 12, 'Q': 13, 'R': 14, 'S': 15, 'T': 16, 'V': 17, 'W': 18, 'Y': 19}

Load tumorCDR3 data

In [4]:
s = []
my_file = open("/content/drive/My Drive/DL/VAE/NormalCDR3.txt", "r")
content_list = my_file.read().splitlines()
content_list=np.array(content_list)
print(content_list)



['CASSLKPNTEAFF' 'CASSAHIANYGYTF' 'CASSPRPNTEAFF' ... 'CASSVDVGYEQYF'
 'CASRLAGQETQYF' 'CASSVVPNTEAFF']


In [ ]:
d= {} # bipphysics dictionary
with open("/content/drive/My Drive/DL/VAE/AAidx_PCA.txt") as f:
    next(f)
    for line in f.readlines():
        line=line.strip().split('\t') #
        AA=line[0]
        tag=0
        values=[]
        for PC in line[1:]:
            values.append(float(PC))
        if tag==1: 
            continue
        d[AA]=values
d # AA


In [6]:
# max length
#comp_seq=data["amino_acid"].tolist()
comp_seq=content_list
print(comp_seq)
max_len=-1 

for AA in comp_seq:
    if len(AA)>max_len:
        max_len=len(AA)
        #res=AA
print(max_len )

['CASSLKPNTEAFF' 'CASSAHIANYGYTF' 'CASSPRPNTEAFF' ... 'CASSVDVGYEQYF'
 'CASRLAGQETQYF' 'CASSVVPNTEAFF']
19


In [7]:
PC_length=len(d['C'])
def AAindexEncoding(Seq):
    length_seq=len(Seq)
    global max_len
    AAE=np.zeros([max_len,15])
    if length_seq<max_len:
        for amino in range(length_seq):
            AA=Seq[amino]# 
            AAE[amino,]=d[AA] # add PC value 
            
        for amino in range(length_seq,max_len):
            AAE[amino,]=np.zeros(15)
    else: 
        for amino in range(length_seq):
            AA=Seq[amino]# 
            AAE[amino,]=d[AA]
        
    #AAE=np.transpose(AAE.astype(np.float32)) # row as PC. and column as AA sequence 
    return AAE 
  
def GetFeatures(file):
    #sequence=file['amino_acid'].tolist()
    #sequence=np.array(sequence)
    #sequence = file.read().splitlines()
    #sequence=np.array(sequence)
    hot_encode=[]
    for seq in file:
        hot_encode.append(AAindexEncoding(seq))
    hot_encode=np.array(hot_encode)
    result=np.array(hot_encode)
    return(result) # dimension: number of sequence [15*length(sequence)]


r1=GetFeatures(content_list)



In [8]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import numpy as np

from tqdm import tqdm
from torchvision.utils import save_image, make_grid
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split

Load data

In [42]:
# trabsform 
kwargs = {'num_workers': 1, 'pin_memory': True} 
r1_transform=torch.from_numpy(r1)
r1_transform=r1_transform.float()
train_ds, test_ds = torch.utils.data.random_split(r1_transform, (30000, 10000))
print(train_ds, test_ds)

train_loader = DataLoader(dataset=train_ds, batch_size=1000)
test_loader  = DataLoader(dataset=test_ds,  batch_size=1000)





<torch.utils.data.dataset.Subset object at 0x7f451c0c2150> <torch.utils.data.dataset.Subset object at 0x7f451c0c20d0>


In [40]:
from torch.utils.data.dataset import TensorDataset
kwargs = {'num_workers': 1, 'pin_memory': True} 
#r1_transform=torch.Tensor(r1)

train_ds, test_ds = torch.utils.data.random_split(r1, (30000, 10000))



train_loader = DataLoader(dataset=train_ds, batch_size=1000)
test_loader  = DataLoader(dataset=test_ds,  batch_size=1000)


In [45]:
# set paramters:
cuda = True
DEVICE = torch.device("cuda" if cuda else "cpu")


batch_size = 1000

x_dim=285 # 19*15
hidden_dim = 256
latent_dim = 128

lr = 1e-3

epochs = 100

In [11]:
class Encoder(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(Encoder, self).__init__()

        self.FC_input = nn.Linear(input_dim, hidden_dim)
        self.FC_input2 = nn.Linear(hidden_dim, hidden_dim)
        self.FC_mean  = nn.Linear(hidden_dim, latent_dim)
        self.FC_var   = nn.Linear (hidden_dim, latent_dim)
        
        self.LeakyReLU = nn.LeakyReLU(0.2)
        
        self.training = True
        
    def forward(self, x):
        h_       = self.LeakyReLU(self.FC_input(x))
        h_       = self.LeakyReLU(self.FC_input2(h_))
        mean     = self.FC_mean(h_)
        log_var  = self.FC_var(h_)                     # encoder produces mean and log of variance 
                                                       #             (i.e., parateters of simple tractable normal distribution "q"
        
        return mean, log_var

In [12]:
class Decoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, output_dim):
        super(Decoder, self).__init__()
        self.FC_hidden = nn.Linear(latent_dim, hidden_dim)
        self.FC_hidden2 = nn.Linear(hidden_dim, hidden_dim)
        self.FC_output = nn.Linear(hidden_dim, output_dim)
        
        self.LeakyReLU = nn.LeakyReLU(0.2)
        
    def forward(self, x):
        h     = self.LeakyReLU(self.FC_hidden(x))
        h     = self.LeakyReLU(self.FC_hidden2(h))
        
        x_hat = torch.sigmoid(self.FC_output(h))
        return x_hat

In [13]:
class Model(nn.Module):
    def __init__(self, Encoder, Decoder):
        super(Model, self).__init__()
        self.Encoder = Encoder
        self.Decoder = Decoder
        
    def reparameterization(self, mean, var):
        epsilon = torch.randn_like(var).to(DEVICE)        # sampling epsilon        
        z = mean + var*epsilon                          # reparameterization trick
        return z
        
                
    def forward(self, x):
        mean, log_var = self.Encoder(x)
        z = self.reparameterization(mean, torch.exp(0.5 * log_var)) # takes exponential function (log var -> var) ]
        x_hat            = self.Decoder(z)
        
        return x_hat, mean, log_var

In [14]:
encoder = Encoder(input_dim=x_dim, hidden_dim=hidden_dim, latent_dim=latent_dim)
decoder = Decoder(latent_dim=latent_dim, hidden_dim = hidden_dim, output_dim = x_dim)

model = Model(Encoder=encoder, Decoder=decoder).to(DEVICE)

In [33]:
from torch.optim import Adam

BCE_loss = nn.BCELoss()

def loss_function(x, x_hat, mean, log_var):
    #reproduction_loss = nn.functional.binary_cross_entropy(x_hat, x, reduction='sum')
    KLD      = - 0.5 * torch.sum(1+ log_var - mean.pow(2) - log_var.exp())

    return reproduction_loss + KLD


optimizer = Adam(model.parameters(), lr=lr)

In [46]:
print("Start training VAE...")
model.train()

for epoch in range(epochs):
    overall_loss = 0
    for batch_idx, x in enumerate(train_loader):
        x = x.view(1000, x_dim)
        x = x.to(DEVICE)

        optimizer.zero_grad()

        x_hat, mean, log_var = model(x)
        loss = loss_function(x, x_hat, mean, log_var)
        
        overall_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        
    print("\tEpoch", epoch + 1, "complete!", "\tAverage Loss: ", overall_loss / (batch_idx*batch_size))
    
print("Finish!!")

Start training VAE...
	Epoch 1 complete! 	Average Loss:  -56331.00468965517
	Epoch 2 complete! 	Average Loss:  -56341.23241379311
	Epoch 3 complete! 	Average Loss:  -56357.144
	Epoch 4 complete! 	Average Loss:  -56396.44510344828
	Epoch 5 complete! 	Average Loss:  -56409.03324137931
	Epoch 6 complete! 	Average Loss:  -56396.169931034485
	Epoch 7 complete! 	Average Loss:  -56424.94606896552
	Epoch 8 complete! 	Average Loss:  -56414.80441379311
	Epoch 9 complete! 	Average Loss:  -56449.91572413793
	Epoch 10 complete! 	Average Loss:  -56456.1875862069
	Epoch 11 complete! 	Average Loss:  -56463.82675862069
	Epoch 12 complete! 	Average Loss:  -56472.17131034483
	Epoch 13 complete! 	Average Loss:  -56495.485655172415
	Epoch 14 complete! 	Average Loss:  -56523.58593103448
	Epoch 15 complete! 	Average Loss:  -56568.054344827586
	Epoch 16 complete! 	Average Loss:  -56648.31862068966
	Epoch 17 complete! 	Average Loss:  -56696.35434482759
	Epoch 18 complete! 	Average Loss:  -56713.16206896552
	Ep